# Benchmark Analysis
Benchmark data is found under /swann/benchmark

In [2]:
# Imports
import csv
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [1]:
# Helper functions


def csv_to_df(file: str):
    """ Takes in a benchmark file and returns as Pandas DataFrame """
    return pd.read_csv(
        f'/swann/benchmark/{file}.csv',
        header = 10
    )